In [11]:
%load_ext autoreload
%autoreload 2

import copy
import os
import sys
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle
import sklearn

import torch

torch.manual_seed(0)

import torchvision
from torchvision import transforms
import torch.nn as nn

sys.path.append('./../')
from src.dataset import VehiclePredictorDataset
from src.utils import train_model, evaluate_model, get_model, visualize_model
#from src.bilinear import BCNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## GPU Check

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Config Variables

In [13]:
root_path = './../'
data_path = os.path.join(root_path, 'data')
dataset_path = os.path.join(data_path, 'VMMRdb')

# modify this line if you want to train a model on top 100/top 200/top 300 most common make_models
with open(os.path.join(data_path, 'make_model_year_most_common_100.pkl'), 'rb') as f:
    target_make_model_year_labels = pickle.load(f)

# load the dataset for some stats
vp_dataset = VehiclePredictorDataset(root_dir=dataset_path, target_make_model_year_labels=target_make_model_year_labels)
num_images = len(vp_dataset)
num_labels = len(vp_dataset.make_model_year_counts)
class_distribution = vp_dataset.make_model_year_counts
print(f"num_images: {num_images}")
print(f"num_labels: {num_labels}")

num_images: 38098
num_labels: 100


## Define the transforms
- Add other transforms here later, if needed. 
- Do we need any specific transforms for train and val?

In [14]:
def get_transform():
    
    transform = transforms.Compose([
        #transforms.Resize((224, 224)),
        #transforms.Resize((448, 448)),
        
        torchvision.transforms.Resize(size=224),
        transforms.RandomHorizontalFlip(),
        #random affine transformation
        torchvision.transforms.CenterCrop(size=224),
        
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return transform

## Instantiate the train, val and test dataloaders

In [15]:
train_split_ratio = 0.8
val_split_ratio = 0.1
test_split_ration = 0.1

# later see if you can have train-specific transforms
dataset = VehiclePredictorDataset(root_dir=dataset_path, target_make_model_year_labels=target_make_model_year_labels, transform=get_transform())

# split dataset in train and val set
dataset_len = len(dataset)
indices = torch.randperm(dataset_len).tolist()
train_split_index = int(train_split_ratio * dataset_len)
val_split_index = train_split_index + int(val_split_ratio * dataset_len)
train_dataset = torch.utils.data.Subset(dataset, indices[0:train_split_index])
val_dataset = torch.utils.data.Subset(dataset, indices[train_split_index:val_split_index])
test_dataset = torch.utils.data.Subset(dataset, indices[val_split_index:])

set_num_workers = 8;
batch_size = 64

# define the dataloaders
dataloader_train = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=set_num_workers,
)

dataloader_val = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=set_num_workers,
)

dataloader_test = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=set_num_workers,
)

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(val_dataset)
}

print(f'train: 0 to {train_split_index}\nval: {train_split_index} to {val_split_index}\ntest: {val_split_index} to {dataset_len-1}')
print(f'dataset_sizes : {dataset_sizes}')

train: 0 to 30478
val: 30478 to 34287
test: 34287 to 38097
dataset_sizes : {'train': 30478, 'val': 3809, 'test': 3809}


## Model config

In [19]:
# check https://pytorch.org/vision/0.8/models.html to experiment with other backbone model
backbone_model = 'bilinear_vgg16_ft'
num_epochs = 30
lr = 0.01

# get the model
model = get_model(num_labels, backbone_model).to(device)
model_name = 'bilinear_vgg16_ft_100classes_25epochs_makemodelyear_acc_0.628.pth'
model.load_state_dict(torch.load(os.path.join(root_path, 'models', model_name)))

# the reason for computing weights is to account for the class imbalance
weight_distribution = 1 / torch.tensor(list(class_distribution.values()))
weight_distribution = weight_distribution / weight_distribution.sum()
weight_distribution = weight_distribution.to(device)

# define the loss function
criterion = nn.CrossEntropyLoss(weight=weight_distribution)

# define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9, weight_decay = 1e-4)

# define the scheduler
#exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

target_labels = 'makemodelyear'
model_info = {
    'backbone': backbone_model,
    'num_classes': f"{num_labels}classes",
    'num_epochs': f"{num_epochs}epochs",
    'labels': target_labels
}
model_name = "_".join([model_info[key] for key in model_info]) + ".pth"
print(f"model_name: {model_name}")

model_name: bilinear_vgg16_ft_100classes_30epochs_makemodelyear.pth


## Train
- Uncomment this line if you want to train again. 
- The trained models are all stored in this [drive folder]( https://drive.google.com/drive/folders/1RXaKgStTFnVRaLvk-eIHEGhwvQp-y1-c?usp=sharing)

In [20]:
train_model(model, model_info, dataset_sizes, dataloader_train, dataloader_val, criterion, optimizer, exp_lr_scheduler, target_labels, num_epochs)

Epoch 0/29
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 477/477 [02:10<00:00,  3.66it/s]


train Loss: 0.0340 Acc: 0.9985 Time: 2m 10s


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:07<00:00,  8.01it/s]


val Loss: 1.0143 Acc: 0.6188 Time: 2m 18s
learning rate: 0.01
Epoch 1/29
----------


...

val Loss: 1.0267 Acc: 0.6314 Time: 67m 34s
learning rate: 1.0000000000000002e-07
Epoch 29/29
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 477/477 [02:10<00:00,  3.66it/s]


train Loss: 0.0200 Acc: 0.9990 Time: 69m 44s


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:07<00:00,  8.10it/s]


val Loss: 1.0285 Acc: 0.6293 Time: 69m 51s
learning rate: 1.0000000000000002e-07
Training complete in 68m 51s
Best val Acc: 0.632449


## Evaluate on test set

In [21]:
# Load saved model
model_name = 'bilinear_vgg16_ft_100classes_30epochs_makemodelyear_acc_0.632.pth'
backbone_model = 'bilinear_vgg16_ft'
target_labels = 'makemodelyear'
model = get_model(num_labels, backbone_model).to(device)
model.load_state_dict(torch.load(os.path.join(root_path, 'models', model_name)))

cm, cr = evaluate_model(model, dataset_sizes, dataloader_test, target_labels, target_make_model_year_labels)

# plot the CM
cm_pd = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_pd = pd.DataFrame(cm_pd, index=target_make_model_year_labels, columns=target_make_model_year_labels)
cr_pd = pd.DataFrame(cr).transpose()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 3811/3811 [00:22<00:00, 169.90it/s]

Test Acc: 0.6301
Average time per image: 0.0059 seconds


In [ ]:
model

- Confusion matrix

In [ ]:
cm_pd

- Classification Report

In [ ]:
cr_pd

- Save/Load the metrics

In [22]:
cm_pd.to_csv(os.path.join(root_path, 'results', 'bilinear', f"confusion_matrix_{model_name}.csv"), index=False)
cr_pd.to_csv(os.path.join(root_path, 'results', 'bilinear', f"classification_report_{model_name}.csv"), index=False)
#cm_pd_df = pd.read_csv(os.path.join(root_path, 'results', 'baseline_methods', f"confusion_matrix_{model_name}.csv"))
#cr_pd_df = pd.read_csv(os.path.join(root_path, 'results', 'baseline_methods', f"classification_report_{model_name}.csv"))

## Visually Inspect performance
- Load a target model to inpect

In [ ]:
# Load saved model
model = get_model(num_labels, backbone_model).to(device)
model.load_state_dict(torch.load(os.path.join(root_path, 'models', model_name)))

- Visualize for a few examples from the test set

In [ ]:
visualize_model(model, dataset, dataloader_test, target_labels, target_make_model_year_labels, num_images=10)